In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train_dir = r'E:/other_data/overhead/training' + "//"
test_dir = r'E:/other_data/overhead/testing' + "//"

In [3]:
classes=pd.read_csv('E:/other_data/overhead/classes.csv')
classes

,class,label,train_count,test_count
0,car,0,888,112
1,harbor,1,889,111
2,helicopter,2,655,82
3,oil_gas_field,3,888,110
4,parking_lot,4,888,112
5,plane,5,889,111
6,runway_mark,6,801,100
7,ship,7,889,111
8,stadium,8,843,105
9,storage_tank,9,889,111


In [4]:
Name=classes['class']
Label=classes['label']

file_reverse_mapping=dict(zip(Label,Name)) 
print(file_reverse_mapping)

{0: 'car', 1: 'harbor', 2: 'helicopter', 3: 'oil_gas_field', 4: 'parking_lot', 5: 'plane', 6: 'runway_mark', 7: 'ship', 8: 'stadium', 9: 'storage_tank'}


In [5]:
trainx0=[]
trainy0=[]
count=0
for file in Name:
    path=os.path.join(train_dir,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(48,48))
        image=img_to_array(image)
        image=image/255.0
        trainx0.append(image)
        trainy0.append(count)
    count=count+1

In [6]:
testx0=[]
testy0=[]
count=0
for file in Name:
    path=os.path.join(test_dir,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(48,48))
        image=img_to_array(image)
        image=image/255.0
        testx0.append(image)
        testy0.append(count)
    count=count+1

In [7]:
trainy2=to_categorical(trainy0)
X_train=np.array(trainx0).reshape(-1,48,48,3)
y_train=np.array(trainy2)
X_test=np.array(testx0).reshape(-1,48,48,3)

In [8]:
trainx,testx,trainy,testy=train_test_split(X_train,y_train,test_size=0.2,random_state=44)

In [9]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

(6815, 48, 48, 3)
(6815, 10)
(1704, 48, 48, 3)
(1704, 10)


In [10]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [11]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(48,48,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False
print(type(pretrained_model3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<class 'tensorflow.python.keras.engine.training.Model'>


In [12]:
print(pretrained_model3)

In [13]:
pretrained_model3.summary()

Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 24, 24, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [14]:
print(dir(pretrained_model3))


['_TF_MODULE_IGNORED_PROPERTIES', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_activity_regularizer', '_add_inbound_node', '_add_unique_metric_name', '_add_variable_with_custom_getter', '_assert_compile_was_called', '_assert_weights_created', '_autocast', '_base_init', '_build_model_with_inputs', '_cache_output_metric_attributes', '_call_accepts_kwargs', '_call_arg_was_passed', '_call_fn_args', '_callable_losses', '_check_call_args', '_check_trainable_weights_consistency', '_checkpoint_dependencies', '_clear_losses', '_collect_input_masks', '_compile_distribution', '_compile_eagerly', '_compile_from_inputs', '_compile_time_distribution_strategy', '_compile_weights_loss_and_weigh

In [15]:
print(pretrained_model3.output_names)

['avg_pool']


In [16]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(10, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [17]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [18]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=20)

Epoch 1/20
213/213 [==============================] - 19s 89ms/step - loss: 1.1696 - acc: 0.6175 - val_loss: 1.3524 - val_acc: 0.5293
Epoch 2/20
213/213 [==============================] - 10s 48ms/step - loss: 0.7993 - acc: 0.7435 - val_loss: 1.2778 - val_acc: 0.5622
Epoch 3/20
213/213 [==============================] - 10s 48ms/step - loss: 0.7387 - acc: 0.7592 - val_loss: 1.3952 - val_acc: 0.5223
Epoch 4/20
213/213 [==============================] - 10s 47ms/step - loss: 0.7000 - acc: 0.7749 - val_loss: 1.3917 - val_acc: 0.5417
Epoch 5/20
213/213 [==============================] - 10s 47ms/step - loss: 0.6640 - acc: 0.7868 - val_loss: 1.4633 - val_acc: 0.5164
Epoch 6/20
213/213 [==============================] - 11s 50ms/step - loss: 0.6627 - acc: 0.7809 - val_loss: 1.5440 - val_acc: 0.4842
Epoch 7/20
213/213 [==============================] - 10s 48ms/step - loss: 0.6676 - acc: 0.7825 - val_loss: 1.4398 - val_acc: 0.5317
Epoch 8/20
213/213 [==============================] - 10s 47ms

In [19]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

              precision    recall  f1-score   support

           0       0.58      0.55      0.57       172
           1       0.76      0.78      0.77       189
           2       0.44      0.71      0.55       128
           3       0.69      0.50      0.58       172
           4       0.47      0.68      0.56       170
           5       0.93      0.14      0.24       190
           6       0.44      0.90      0.59       174
           7       0.93      0.46      0.62       193
           8       0.77      0.39      0.52       170
           9       0.53      0.74      0.62       146

    accuracy                           0.58      1704
   macro avg       0.65      0.59      0.56      1704
weighted avg       0.67      0.58      0.56      1704



In [20]:
ONNX_test_pic = X_train[:1]
print(ONNX_test_pic.shape)

ONNX_test_model_res=model.predict(ONNX_test_pic)
print('modl Predicted:',ONNX_test_model_res)

(1, 48, 48, 3)
modl Predicted: [[1.0932939e-01 2.4644444e-03 4.0585441e-03 6.7247183e-04 6.3516228e-03
  1.5654268e-05 2.3421229e-04 8.4816122e-05 6.0461899e-03 8.7074268e-01]]


5.儲存模型

In [21]:
# 方法1 用 model.save 
# 這樣只有儲存架構而已??
# save_model_name = "overhead_mnist.h5"
save_model_name = "/pb/overhead_mnist_model_save.pb"
# save_model_name = "/pb/overhead_mnist_model_save.pbtxt"
model.save('./' + save_model_name)
print('已儲存 '+save_model_name)

已儲存 /pb/overhead_mnist_model_save.pb


In [22]:
# # 方法2 用 graph_def.SerializeToString
# # fail 未來會廢棄 然後會跳錯
# with tf.Graph().as_default() as graph :
#      graph_def = graph.as_graph_def()
#      with tf.gfile.FastGFile("/pb/overhead_mnist_FastGFile.pb", mode='wb') as f:
#           f.write(graph_def.SerializeToString())

#      with tf.io.gfile.GFile("/pb/overhead_mnist_GFile.pb", "wb") as f:
#           f.write(graph_def.SerializeToString())

In [23]:
# # 方法3 用 write_graph
# # 看起來也是錯的 因為很小 然後netron也無法讀取
# tf.io.write_graph(tf.get_default_graph(), "./pb/", "test_pb_model.pb", as_text=False)

In [24]:
# 測試看看能不能印出 node 的資訊
# fail 無法 數量都是 0
with tf.Graph().as_default() as graph :
    print("len(graph.get_operations()) : ",len(graph.get_operations()))
    for op in graph.get_operations():
        print(op.name, op.values())
    
    graph_def = graph.as_graph_def()
    print("graph_def.node : ",len(graph_def.node))
    for node in graph_def.node:
        print(node)

len(graph.get_operations()) :  0
graph_def.node :  0


In [25]:
# # # 方法4 frozen_graph
# # 因為上一步無法取得 node 的資訊，所以我也不知道 output layer 名稱
#     # 所以會跳錯
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.local_variables_initializer())

#     frozen_graph = tf.graph_util.convert_variables_to_constants(
#         sess, tf.get_default_graph().as_graph_def(), [output layer 名稱])

#     tf.io.write_graph(frozen_graph, "./pb/", "frozen_model.pb", as_text=False)
#     tf.io.write_graph(frozen_graph, "./pb/", "frozen_model.pbtxt", as_text=True)

In [26]:
# 把 tensorflow 的 model 存成 .onnx 的檔案
import tf2onnx

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = "./ONNX_model/" + "DenseNet201_OverHead_MNIST" + ".onnx"
print("output_path : ", output_path)
# model 產出在上一層的 ONNX_model 資料夾

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

output_path :  ./ONNX_model/DenseNet201_OverHead_MNIST.onnx

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 1021 variables.
INFO:tensorflow:Converted 1021 variables to const ops.



Removing optional input keras_learning_phase that appears to be a keras learning phase parameter. Use --ignore_default to force this into an input.
Removing optional input keras_learning_phase that appears to be a keras learning phase parameter. Use --ignore_default to force this into an input.


In [27]:
# 把剛剛存的 .onnx 的檔案，讀取並執行辨識
    # 但是會跳 exception...
    # 不過目前先確定有產出 ONNX 的檔案就好了，反正也不會在 Python 裡面讀取
import onnxruntime as rt
providers = ['CPUExecutionProvider']
# output_path = "./ONNX_model/" + "DenseNet201_OverHead_MNIST_same" + ".onnx"
print("output_path :",output_path)
m = rt.InferenceSession(output_path, providers=providers)
inname = [input_setting.name for input_setting in m.get_inputs()]
outname = [input_setting.name for input_setting in m.get_outputs()]
print('in name', inname)
print('out name', outname)
onnx_pred = m.run(outname, {"input_1": ONNX_test_pic})
print('ONNX Predicted:', onnx_pred[:1])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(ONNX_test_model_res, onnx_pred[0], rtol=1e-4) # 1e-5 會有兩個不一樣

output_path : ./ONNX_model/DenseNet201_OverHead_MNIST.onnx
in name ['input_1']
out name ['dense_1']
ONNX Predicted: [array([[1.0932974e-01, 2.4644448e-03, 4.0586013e-03, 6.7247299e-04,
        6.3516852e-03, 1.5654317e-05, 2.3421572e-04, 8.4816631e-05,
        6.0462258e-03, 8.7074208e-01]], dtype=float32)]


In [28]:
# 我的電腦到這裡還可以執行

In [29]:
print(dir(his))
print(dir(his.history))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chief_worker_only', '_keras_api_names', '_keras_api_names_v1', 'epoch', 'history', 'model', 'on_batch_begin', 'on_batch_end', 'on_epoch_begin', 'on_epoch_end', 'on_predict_batch_begin', 'on_predict_batch_end', 'on_predict_begin', 'on_predict_end', 'on_test_batch_begin', 'on_test_batch_end', 'on_test_begin', 'on_test_end', 'on_train_batch_begin', 'on_train_batch_end', 'on_train_begin', 'on_train_end', 'params', 'set_model', 'set_params', 'validation_data']
['__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_

In [30]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

KeyError: 'accuracy'

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
path0='../input/overheadmnist/overhead/testing/runway_mark/08874.jpg'
load_img(path0,target_size=(300,300))

In [ ]:
image=load_img(path0,target_size=(48,48))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=file_reverse_mapping[value]
print("Prediction is {}.".format(move_name))

In [ ]:
pred2=model.predict(X_test)
print(pred2[0:10])

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]
print(PRED[0:10])

In [ ]:
ANS=testy0
ANS[0:10]

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)